# Politician Sentiment Analysis - Classification

Name:       Devin Patel  
Class:      CS 588 - 01  
Term:       FA 22  
Project:    Determining party alignment based on 2016 Election tweets.  
File Purpose: To gauge various classifier accuracies on selected features.

## Initialization

In [2]:
# Imports
import numpy as np
import pandas as pd
import scipy.io
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split    # Splits sample set into train/test sets
from sklearn.model_selection import cross_val_score     # Used to evaluate classifier performance

from sklearn.model_selection import StratifiedKFold     # Splits a matrix into K sections
from sklearn.naive_bayes import GaussianNB              # Gaussian Naive Bayes
from sklearn.svm import SVC                             # SVM
from sklearn.neural_network import MLPClassifier        # CNN
from sklearn.pipeline import Pipeline                   # For estimating classifier models
from sklearn.metrics import ConfusionMatrixDisplay      # For gauging classifier model accuracy

import os                                               # For exporting figures
import dataframe_image as dfi
import json


# Paths and Constants
twitter_data_path = r"Data/TweetData.pkl"
RANDOM_STATE = 12
KEPT_FEATURES = 2
NORM_RANGE = (0, 1)


In [3]:
# Exports pyplot figure
import os

def exportFig(fname):
    exportPath = "Figures/dimension_reduction"

    try:
        if not os.path.exists(exportPath): os.mkdir(exportPath)
    except Exception:
        print("Can't create a directory to store figures, so they will not be saved.")
        return
    
    exportPath = os.path.join(exportPath, fname)
    
    plt.savefig(exportPath)

In [4]:
# Loading data from pkl file
X_df = pd.read_pickle(twitter_data_path, compression='infer')

# Separate ground truth from dataframe.
party_gth_df = pd.DataFrame(X_df.pop('Party').astype('int'))

# Create class labels list for both ground truth dataframes
party_gth_labels = ["R", "D"] # Based on preprocessing: 0 = Republican, 1 = Democrat

# Create feature labels list
feature_labels = X_df.columns.values.tolist()

scaler_model = MinMaxScaler(feature_range=NORM_RANGE)
scaler_model.fit(X_df)
X = scaler_model.transform(X_df)
X_df = pd.DataFrame(X, columns=feature_labels)

# Data is ready
print(f"\nShape of main dataframe: {X_df.shape}")



Shape of main dataframe: (65952, 4)


In [ ]:
# Initialize feature selection here
